<a href="https://colab.research.google.com/github/rogerwzeng/e17/blob/main/DGMD_PSET4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Set 4 (75 pts)

In this problem set, you'll explore topics related to odometry and wheel encoders. Unless specified otherwise, we will be working with a two wheeled differential drive robot with incremental optical encoders.

**Note**: the numbers used may not accurately reflect real life circumstances - they were chosen to be mathematically convenient. Furthermore, in practice, wheel odometry can be very noisy and has to often be paired with other sensors for higher accuracy localization.

**This problem set is all short-response / calculations, thus in terms of feedback from GradeScope, it will be very limited**. This is to avoid simply guessing and checking your answer. However, to provide some assistance on whether you're on the right track, for some of the questions, GradeScope will provide information on correctness. For example, GradeScope will tell you whether you got the right values for time $t=5$ - since the questions build up on one another, if you successfully derived the correct answer for a question / quantity at time $t=5$, then it's a good sign (**but not a guarantee**) that your answers for time $t = \{1,2,3,4\}$ are probably correct (as the answers for time $t=5$ would most likely need the correct calculations to that point).

**In terms of numeric precision, please have 4 decimal point precision (if applicable) (e.g., 3.1415 has 4 decimal point precision). Additionally, only provide the numeric answer (do not include any units or other text in the answer field).**


In [ ]:
### DO NOT CHANGE ###
# importing the libraries (do not remove or add libraries)
from typing import List, Set, Dict, Tuple, Optional
import numpy as np

## Robot Directional Motion (10 pts)

You have a two wheeled differential drive robot. On each motor is an incremental optical encoder. The optical encoder provides information on the rotation of the motor.

We can use the data from the encoders to determine the speed of each wheel. In this section, match the different scenarios of wheel speeds with their corresponding motion. For this problem set, the left and right direction of the robot is defined absolutely by the frontward orientation of the robot (i.e., imagine looking in the direction that is the robot's forward direction, that left will always be left regardless of whether robot is going forward or backwards); refer to the diagram below where the forward edge is indicated by the blue triangle indicators. Each question is 1 point.

![](https://drive.google.com/uc?export=view&id=14r7MpVtQtlCoBGqMQtC0XtKyjXhSW5ze)

In [ ]:
#@title Wheel Speed Scenarios

options = ["", "Forward Left", "Forward Straight", "Forward Right", "Backward Left", "Backward Straight", "Backward Right", "Stationary", "Spinning in Place"]

#@markdown #### Scenario1 (1 pt)
#@markdown The left wheel and right wheel are both moving forward, with the left wheel moving faster than the right
scenario1 = "Forward Right" #@param ["", "Forward Left", "Forward Straight", "Forward Right", "Backward Left", "Backward Straight", "Backward Right", "Stationary", "Spinning in Place"]

#@markdown #### Scenario2 (1 pt)
#@markdown The left and right wheel are moving at same speed, but in opposite directions of each other
scenario2 = "Spinning in Place" #@param ["", "Forward Left", "Forward Straight", "Forward Right", "Backward Left", "Backward Straight", "Backward Right", "Stationary", "Spinning in Place"]

#@markdown #### Scenario3 (1 pt)
#@markdown The left and right wheel are moving at same speed in the forward direction
scenario3 = "Forward Straight" #@param ["", "Forward Left", "Forward Straight", "Forward Right", "Backward Left", "Backward Straight", "Backward Right", "Stationary", "Spinning in Place"]

#@markdown #### Scenario4 (1 pt)
#@markdown The left wheel is moving backwards and the right wheel is moving forward. The speed of the right wheel is greater than the speed of the left wheel
scenario4 = "Backward Left" #@param ["", "Forward Left", "Forward Straight", "Forward Right", "Backward Left", "Backward Straight", "Backward Right", "Stationary", "Spinning in Place"]

#@markdown #### Scenario5 (1 pt)
#@markdown  The left and right wheel are moving at same speed in the backward direction
scenario5 = "Backward Straight" #@param ["", "Forward Left", "Forward Straight", "Forward Right", "Backward Left", "Backward Straight", "Backward Right", "Stationary", "Spinning in Place"]

#@markdown #### Scenario6 (1 pt)
#@markdown  The left wheel is moving faster than the right wheel in the opposite direction. The right wheel is spinning forward
scenario6 = "Backward Right" #@param ["", "Forward Left", "Forward Straight", "Forward Right", "Backward Left", "Backward Straight", "Backward Right", "Stationary", "Spinning in Place"]

#@markdown #### Scenario7 (1 pt)
#@markdown  Both wheels are not spinning
scenario7 = "Stationary" #@param ["", "Forward Left", "Forward Straight", "Forward Right", "Backward Left", "Backward Straight", "Backward Right", "Stationary", "Spinning in Place"]

#@markdown #### Scenario8 (1 pt)
#@markdown  Only the left wheel is spinning forward and the surface is very smooth (meaning the other wheel will glide in place)
scenario8 = "Forward Right" #@param ["", "Forward Left", "Forward Straight", "Forward Right", "Backward Left", "Backward Straight", "Backward Right", "Stationary", "Spinning in Place"]

#@markdown #### Scenario9 (1 pt)
#@markdown  Only the left wheel is spinning forward and the surface is very rough (meaning the other wheel will have a lot of friction to overcome and can not move)
scenario9 = "Spinning in Place" #@param ["", "Forward Left", "Forward Straight", "Forward Right", "Backward Left", "Backward Straight", "Backward Right", "Stationary", "Spinning in Place"]

#@markdown #### Scenario10 (1 pt)
#@markdown  Both wheels are moving backward. The right wheel is spinning faster than the left wheel
scenario10 = "Backward Left" #@param ["", "Forward Left", "Forward Straight", "Forward Right", "Backward Left", "Backward Straight", "Backward Right", "Stationary", "Spinning in Place"]


## Incremental Optical Encoder Model (20 pts)

In this section, you'll explore the data from the incremental optical encoders located on the motors for the wheels. An incremental optical encoder functions by having an electric circuit that is connected to a photo sensor which influences the amount of voltage in the circuit. There is a disk that separates the circuit with the photo sensor from a light emitting diode (LED). The disk is comprised of slits and rotates with the motor. As the disk rotates, light passes through the slits which will be detected by the photo sensor and affect the voltage of the circuit, thus capturing the rotation. The slits are equally spaced and represent some fixed unit of rotation.

![](https://drive.google.com/uc?export=view&id=12C2DbfEN38d6dnTLzIYY12V5Bzk9TZm6)

For notation purposes, unless specified otherwise, we'll be using subscript $_L$ and subscript $_R$ to denote left and right, respectively.

You collect 5 measurements of data from the incremental optical encoder at each wheel, starting at time $t=0$ to time $t=5$. Each wheel encoder keeps track of the total number of slits that the encoder has passed. Thus, the measurement at each time step represents the total number of slits seen up to that time step. The total number of slits will increase in value for counter clockwise rotation, which translates to moving in the forward direction - in this scenario, the distance of the wheel will be positive. Whereas when the rotation is clockwise, this translates to backward motion and will result in the total number of slits decreasing - in this scenario, the distance of the wheel will be negative.

Here are the measurements from each encoder, with $E_L$ representing the encoder measurements from the motor for the left wheel and $E_R$ representing the encoder measurements from the motor for the right wheel:

\begin{align}
E_L &= \begin{bmatrix}0 & \text{ }48 & \text{ }30 & \text{ }33\text{ } & \text{ }21\text{ } & \text{ }51\text{ } \end{bmatrix} \\
E_R &= \begin{bmatrix}0 & \text{ }12 & \text{ }72 & 135 & 171 & 201 \end{bmatrix}
\end{align}


For our robot, the circumference of the left encoder, $C_L$, and the circumference of the right encoder, $C_R$, will both be equal to $6$. The total number of slits on the left encoder, $T_L$, and the total number of slits on the right encoder, $T_R$, will both be equal to $72$.

Let $d_{i, t}$ be the distance travelled by wheel $i$ at time $t$. Determine the values for $d_{L, t}$ and $d_{R, t}$ for $t = \{1,2,3,4,5\}$. In laymans terms, find the distance travelled by the left wheel and the right wheel at each time step from $t=1$ to $t=5$.

**Hint**: Let's define $s_{i,t}$ to be the total number of slits passed by the encoder for wheel $i$ at time $t$. We can determine the amount of distance of wheel $i$ at time $t$ by this formula:
$$
d_{i, t} = \frac{s_{i, t} - s_{i, t-1}}{T_i}C_i
$$

In [ ]:
#@title Left Optical Encoder (10 pts)

#@markdown **In terms of numeric precision, please have 4 decimal point precision (if applicable) (e.g., 3.1415 has 4 decimal point precision). Additionally, only provide the numeric answer (do not include any units or other text in the answer field).**

#@markdown Left wheel distance at time $t = 1$ (2 pts)
distance_left_wheel_time_1 = "" #@param {type: "string"}

#@markdown Left wheel distance at time $t = 2$ (2 pts)
distance_left_wheel_time_2 = "" #@param {type: "string"}

#@markdown Left wheel distance at time $t = 3$ (2 pts)
distance_left_wheel_time_3 = "" #@param {type: "string"}

#@markdown Left wheel distance at time $t = 4$ (2 pts)
distance_left_wheel_time_4 = "" #@param {type: "string"}

#@markdown Left wheel distance at time $t = 5$ (2 pts)
distance_left_wheel_time_5 = "" #@param {type: "string"}


In [ ]:
#@title Right Optical Encoder (10 pts)

#@markdown **In terms of numeric precision, please have 4 decimal point precision (if applicable) (e.g., 3.1415 has 4 decimal point precision). Additionally, only provide the numeric answer (do not include any units or other text in the answer field).**

#@markdown Right wheel distance at time $t = 1$ (2 pts)
distance_right_wheel_time_1 = "" #@param {type: "string"}

#@markdown Right wheel distance at time $t = 2$ (2 pts)
distance_right_wheel_time_2 = "" #@param {type: "string"}

#@markdown Right wheel distance at time $t = 3$ (2 pts)
distance_right_wheel_time_3 = "" #@param {type: "string"}

#@markdown Right wheel distance at time $t = 4$ (2 pts)
distance_right_wheel_time_4 = "" #@param {type: "string"}

#@markdown Right wheel distance at time $t = 5$ (2 pts)
distance_right_wheel_time_5 = "" #@param {type: "string"}

## Wheel Odometry Model - Relative Motion (20 pts)

For our wheel odometry model, we'll treat our robot as a singular point that is located equidistant between the two, parallel wheels. This singular point will serve as the reference point for the robot. The distance between the reference point and a wheel is $d_w = 2$.

![](https://drive.google.com/uc?export=view&id=1M2B4nShthiEGCUWwVly4b1ni_49GCSgC)

Using the encoder values, $E_L$ and $E_R$, from the previous section, determine the distance travelled by the reference point for time $t=1$ to time $t=5$. Additionally, determine the $\Delta \theta$, **in radians**, for time $t=1$ to time $t=5$.

**Hint**: refer to the odometry model covered in this article: [Wheel Odometry Model for Differential Drive Robotics](https://medium.com/@nahmed3536/wheel-odometry-model-for-differential-drive-robotics-91b85a012299)

In [ ]:
#@title Reference Point Distance (10 pts)

#@markdown **In terms of numeric precision, please have 4 decimal point precision (if applicable) (e.g., 3.1415 has 4 decimal point precision). Additionally, only provide the numeric answer (do not include any units or other text in the answer field).**

#@markdown Reference point distance at time $t = 1$ (2 pts)
distance_reference_point_time_1 = "" #@param {type: "string"}

#@markdown Reference point distance at time $t = 2$ (2 pts)
distance_reference_point_time_2 = "" #@param {type: "string"}

#@markdown Reference point distance at time $t = 3$ (2 pts)
distance_reference_point_time_3 = "" #@param {type: "string"}

#@markdown Reference point distance at time $t = 4$ (2 pts)
distance_reference_point_time_4 = "" #@param {type: "string"}

#@markdown Reference point distance at time $t = 5$ (2 pts)
distance_reference_point_time_5 = "" #@param {type: "string"}


In [ ]:
#@title $\Delta \theta$ (10 pts)

#@markdown **In terms of numeric precision, please have 4 decimal point precision (if applicable) (e.g., 3.1415 has 4 decimal point precision). Additionally, only provide the numeric answer (do not include any units or other text in the answer field).**

#@markdown $\Delta \theta$ at time $t = 1$ (2 pts)
delta_theta_time_1 = "" #@param {type: "string"}

#@markdown $\Delta \theta$ at time $t = 2$ (2 pts)
delta_theta_time_2 = "" #@param {type: "string"}

#@markdown $\Delta \theta$ at time $t = 3$ (2 pts)
delta_theta_time_3 = "" #@param {type: "string"}

#@markdown $\Delta \theta$ at time $t = 4$ (2 pts)
delta_theta_time_4 = "" #@param {type: "string"}

#@markdown $\Delta \theta$ at time $t = 5$ (2 pts)
delta_theta_time_5 = "" #@param {type: "string"}


## Wheel Odometry Model - Absolute Position (20 pts)

In the previous section, we calculated the relative change in distance and angle for the robot between time steps. We can translate this to an absolute position if the start position and orientation is know.

In this section, let's model the robot's terrain as a coordinate plane. At time $t=0$, the robot is at position $(0,0)$ facing forward in the right / positive `x`-direction (more precisely, has absolute orientation of 0 radians). Based on this initial starting point and orientation, determine the absolute position of the robot at each time step from time $t=1$ to time $t=5$. Note: the numbers for this section might be more "messy" because trig functions are getting involved.

**Hint**: We can model the dynamics of the robot with:

$$
\begin{align}
\begin{bmatrix}
           x_{t}\\
           y_{t}\\
           \theta_{t}
    \end{bmatrix} =
    \begin{bmatrix}
           x_{t-1}\\
           y_{t-1}\\
           \theta_{t-1}
    \end{bmatrix} +
    \begin{bmatrix}
           d_t\cos(\theta_{t-1} + \frac{\Delta\theta_t}{2})\\
           d_t\sin(\theta_{t-1} + \frac{\Delta\theta_t}{2})\\
           \Delta \theta_t
    \end{bmatrix}
\end{align}
$$

where $d_t$ is the distance travelled by the reference point at time $t$ and $\Delta \theta_t$ is the $\Delta \theta$ or rotation at time $t$. For more details, refer to the odometry model covered in this article: [Wheel Odometry Model for Differential Drive Robotics](https://medium.com/@nahmed3536/wheel-odometry-model-for-differential-drive-robotics-91b85a012299). Also make sure to use the right units (i.e., radians or degrees).


In [ ]:
#@title Absolute Position

#@markdown **In terms of numeric precision, please have 4 decimal point precision (if applicable) (e.g., 3.1415 has 4 decimal point precision). Additionally, only provide the numeric answer (do not include any units or other text in the answer field).**

#@markdown Reference point $(x, y)$ coordinate at time $t = 1$ (4 pts - the $x$ and $y$ are each 2 points)
x_coordinate_absolute_position_time_1 = "" #@param {type: "string"}
y_coordinate_absolute_position_time_1 = "" #@param {type: "string"}

#@markdown Reference point $(x, y)$ coordinate at time $t = 2$ (4 pts - the $x$ and $y$ are each 2 points)
x_coordinate_absolute_position_time_2 = "" #@param {type: "string"}
y_coordinate_absolute_position_time_2 = "" #@param {type: "string"}

#@markdown Reference point $(x, y)$ coordinate at time $t = 3$ (4 pts - the $x$ and $y$ are each 2 points)
x_coordinate_absolute_position_time_3 = "" #@param {type: "string"}
y_coordinate_absolute_position_time_3 = "" #@param {type: "string"}

#@markdown Reference point $(x, y)$ coordinate at time $t = 4$ (4 pts - the $x$ and $y$ are each 2 points)
x_coordinate_absolute_position_time_4 = "" #@param {type: "string"}
y_coordinate_absolute_position_time_4 = "" #@param {type: "string"}

#@markdown Reference point $(x, y)$ coordinate at time $t = 5$ (4 pts - the $x$ and $y$ are each 2 points)
x_coordinate_absolute_position_time_5 = "" #@param {type: "string"}
y_coordinate_absolute_position_time_5 = "" #@param {type: "string"}

## EXTRA CREDIT: Wheel Odometry Model - Absolute Orientation (10 pts)

**Note**: This section is optional extra credit. This section is intended for students looking for more depth or advanced material. This section may require additional self-study to complete. Since this is extra credit, assistance from the autograder will not be provided. Furthermore, there will be limited support from course staff, office hours, and Ed discussion.

In the previous section, we determined the absolute position of the robot relative to our starting point. In this section, use the information provided so far to determine the orientation of the robot for time $t = 1$ to time $t = 5$. The orientation of the robot is the unit vector along the forward direction of robot. For example, the orientation for the robot at time $t = 0$, the starting position, is:

$$
  \begin{bmatrix}
           1\\
           0
    \end{bmatrix}
$$

In [ ]:
#@title Absolute Orientation

#@markdown **In terms of numeric precision, please have 4 decimal point precision (if applicable) (e.g., 3.1415 has 4 decimal point precision). Additionally, only provide the numeric answer (do not include any units or other text in the answer field).**

#@markdown Absolute orientation at time $t = 1$ (2 pts - the $x$ and $y$ are each 1 points)
x_component_absolute_orientation_time_1 = "" #@param {type: "string"}
y_component_absolute_orientation_time_1 = "" #@param {type: "string"}

#@markdown Absolute orientation at time $t = 2$ (2 pts - the $x$ and $y$ are each 1 points)
x_component_absolute_orientation_time_2 = "" #@param {type: "string"}
y_component_absolute_orientation_time_2 = "" #@param {type: "string"}

#@markdown Absolute orientation at time $t = 3$ (2 pts - the $x$ and $y$ are each 1 points)
x_component_absolute_orientation_time_3 = "" #@param {type: "string"}
y_component_absolute_orientation_time_3 = "" #@param {type: "string"}

#@markdown Absolute orientation at time $t = 4$ (2 pts - the $x$ and $y$ are each 1 points)
x_component_absolute_orientation_time_4 = "" #@param {type: "string"}
y_component_absolute_orientation_time_4 = "" #@param {type: "string"}

#@markdown Absolute orientation at time $t = 5$ (2 pts - the $x$ and $y$ are each 1 points)
x_component_absolute_orientation_time_5 = "" #@param {type: "string"}
y_component_absolute_orientation_time_5 = "" #@param {type: "string"}

## Problem Set Survey (5 pts)

Please fill out the survey questions (the first five are each worth 1 point; the last one is optional).


1.   `TIME` (1 pt): approximately how many hours did you spend on the problem set? Please use decimals to express partial hours (e.g., a value of `2.5` means two and half hours).
2.   `DIFFICULTY` (1 pt): on a scale of 1-10, how difficult was this problem set with 1 being very easy and 10 being very hard?
3.   `FAVORITE_PART` (1 pt): What was your favorite topic / learning from the unit (i.e., between the last pset and this pset)? This should contain at least 10 words.
4.   `WENT_WELL` (1 pt): What went well? Describe what went well with the course so far (this can be about the lecture, assignments, and/or other course content). This should contain at least 10 words.
5.   `CHALLENGING` (1 pt): What was challenging? Describe what was challenging or didn't go well with the course so far (this can be about the lecture, assignments, and/or other course content). This should contain at least 10 words.
6.   `COMMENTARY` (0 pt): If there is anything else you'd like to share with course staff, please add it here. If not, no need to change / edit the default string.


In [ ]:
#@title Problem Set Survey Questions
TIME = "" #@param {type:"string"}

DIFFICULTY = "" #@param ["", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]

FAVORITE_PART = "" #@param {type:"string"}

WENT_WELL = "" #@param {type:"string"}

CHALLENGING = "" #@param {type:"string"}

COMMENTARY = "" #@param {type:"string"}

**<font color='red'>To submit, please download as a Python (.py) file and submit on Gradescope (navigate to File > Download > Download .py). Please use the correct file name and comment out any test / extraneous code to avoid any compile and parser issues </font>**